# Text Generation with Adjustable Parameters

This notebook demonstrates how different parameters affect text generation, including temperature, max token length, and sampling methods.


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F

from transformers import GPT2LMHeadModel, GPT2Tokenizer, set_seed
import matplotlib.pyplot as plt
import pandas as pd



import ipywidgets as widgets



from IPython.display import display, HTML, clear_output



import warnings



warnings.filterwarnings("ignore")



set_seed(42)



# Load model and tokenizer



print("Loading GPT-2 model...")



tokenizer = GPT2Tokenizer.from_pretrained("gpt2")



model = GPT2LMHeadModel.from_pretrained("gpt2")



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



model = model.to(device)

print(f"Model loaded! Using {device}")

Loading GPT-2 model...
Model loaded! Using cpu


## Understanding Generation Parameters

Key parameters that affect text generation:


In [4]:
parameters = [
    [
        "Temperature",
        "Controls randomness. Higher values (>1.0) make output more random, lower values (0.2-0.5) make it more focused.",
    ],
    ["Max Length", "Maximum number of tokens to generate."],
    ["Top-k", "Only sample from the top k most likely next tokens."],
    [
        "Top-p",
        "Sample from the smallest set of tokens whose cumulative probability exceeds probability p.",
    ],
    ["Repetition Penalty", "Penalizes repeated tokens to reduce repetition."],
]
display(pd.DataFrame(parameters, columns=["Parameter", "Description"]))

,Parameter,Description
0,Temperature,Controls randomness. Higher values (>1.0) make...
1,Max Length,Maximum number of tokens to generate.
2,Top-k,Only sample from the top k most likely next to...
3,Top-p,Sample from the smallest set of tokens whose c...
4,Repetition Penalty,Penalizes repeated tokens to reduce repetition.


## Token Probability Visualization


In [5]:
def get_token_probabilities(input_text, top_k=10):
    """Get probability distribution for the next token after the input text."""
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits

    next_token_logits = logits[0, -1, :]
    next_token_probs = F.softmax(next_token_logits, dim=0)

    topk_probs, topk_indices = torch.topk(next_token_probs, top_k)
    topk_tokens = [tokenizer.decode([idx.item()]) for idx in topk_indices]

    return topk_tokens, topk_probs.cpu().numpy()


def plot_token_probabilities(tokens, probs, temperature=1.0):
    """Plot token probabilities with temperature adjustment."""
    if temperature != 1.0:
        # Simulate temperature effect
        probs_temp = probs ** (1.0 / temperature)
        probs_temp = probs_temp / probs_temp.sum()  # Renormalize
    else:
        probs_temp = probs

    plt.figure(figsize=(10, 5))
    x = np.arange(len(tokens))
    width = 0.35

    if temperature != 1.0:
        plt.bar(x - width / 2, probs, width, label="Original", color="blue", alpha=0.7)
        plt.bar(
            x + width / 2, probs_temp, width, label=f"Temp={temperature}", color="red", alpha=0.7
        )
        plt.legend()
    else:
        plt.bar(x, probs, width, color="blue", alpha=0.7)

    plt.title(
        f'Next Token Probabilities{" (Temperature = " + str(temperature) + ")" if temperature != 1.0 else ""}',
        fontsize=14,
    )
    plt.xlabel("Tokens")
    plt.ylabel("Probability")
    plt.xticks(x, tokens, rotation=45, ha="right")
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()

    return probs_temp

## Text Generation with Parameters


In [ ]:
def generate_text(
    prompt, max_length=50, temperature=1.0, top_k=0, top_p=0.9, repetition_penalty=1.0
):
    """Generate text with adjustable parameters."""


    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)


    # Generate text


    output_sequences = model.generate(
        input_ids,

        max_length=len(input_ids[0]) + max_length,
        temperature=temperature,

        top_k=top_k,
        top_p=top_p,

        repetition_penalty=repetition_penalty,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
    )


    # Decode the output


    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)


    # Extract just the generated part (after the prompt)


    if generated_text.startswith(prompt):


        generated_text = generated_text[len(prompt) :]

    return generated_text

## Interactive Text Generation Interface


In [7]:
# Create widgets for parameter controls
prompt_input = widgets.Textarea(
    value="The future of AI technology will",
    placeholder="Enter your prompt here...",
    description="Prompt:",
    layout=widgets.Layout(width="90%", height="80px"),
)

temperature_slider = widgets.FloatSlider(
    value=0.7,
    min=0.1,
    max=2.0,
    step=0.1,
    description="Temperature:",
    continuous_update=False,
    layout=widgets.Layout(width="50%"),
)

max_length_slider = widgets.IntSlider(
    value=30,
    min=10,
    max=100,
    step=5,
    description="Max Tokens:",
    continuous_update=False,
    layout=widgets.Layout(width="50%"),
)

top_k_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=50,
    step=5,
    description="Top-K:",
    continuous_update=False,
    layout=widgets.Layout(width="50%"),
)

top_p_slider = widgets.FloatSlider(
    value=0.9,
    min=0.1,
    max=1.0,
    step=0.05,
    description="Top-P:",
    continuous_update=False,
    layout=widgets.Layout(width="50%"),
)

repetition_slider = widgets.FloatSlider(
    value=1.2,
    min=1.0,
    max=2.0,
    step=0.1,
    description="Rep. Penalty:",
    continuous_update=False,
    layout=widgets.Layout(width="50%"),
)

show_probs_checkbox = widgets.Checkbox(
    value=True,
    description="Show token probabilities",
)

generate_button = widgets.Button(description="Generate Text", button_style="success", icon="play")

output = widgets.Output()
status = widgets.HTML(value="")


def on_generate_button_clicked(b):
    with output:
        clear_output()
        status.value = "<i>Generating...</i>"

        prompt = prompt_input.value.strip()
        if not prompt:
            status.value = "<span style='color:red'>Please enter a prompt</span>"
            return

        try:
            # Generate text
            generated_text = generate_text(
                prompt,
                max_length=max_length_slider.value,
                temperature=temperature_slider.value,
                top_k=top_k_slider.value,
                top_p=top_p_slider.value,
                repetition_penalty=repetition_slider.value,
            )

            # Display results
            display(HTML(f"<h3>Generated Text:</h3>"))
            display(
                HTML(
                    f"<p><b>Prompt:</b> <span style='background-color:#e6f7ff'>{prompt}</span></p>"
                )
            )
            display(
                HTML(
                    f"<p><b>Completion:</b> <span style='background-color:#f6ffe6'>{generated_text}</span></p>"
                )
            )

            # Parameter summary
            params = f"Temperature={temperature_slider.value}, Top-K={top_k_slider.value if top_k_slider.value > 0 else 'Off'}, Top-P={top_p_slider.value}, Repetition Penalty={repetition_slider.value}"
            display(HTML(f"<p><b>Parameters:</b> {params}</p>"))

            # Show token probabilities if checked
            if show_probs_checkbox.value:
                tokens, probs = get_token_probabilities(prompt, top_k=10)
                display(HTML("<h4>Next Token Probabilities:</h4>"))
                plot_token_probabilities(tokens, probs, temperature=temperature_slider.value)

            status.value = "<span style='color:green'>Generation complete!</span>"

        except Exception as e:
            status.value = f"<span style='color:red'>Error: {str(e)}</span>"


# Connect the button to the function
generate_button.on_click(on_generate_button_clicked)

# Create layout
display(HTML("<h2>🔮 Text Generation with Parameter Tuning</h2>"))
display(prompt_input)
display(
    widgets.VBox(
        [
            widgets.HBox([temperature_slider, max_length_slider]),
            widgets.HBox([top_k_slider, top_p_slider]),
            widgets.HBox([repetition_slider, show_probs_checkbox]),
        ]
    )
)
display(generate_button)
display(status)
display(output)

Textarea(value='The future of AI technology will', description='Prompt:', layout=Layout(height='80px', width='…

Button(button_style='success', description='Generate Text', icon='play', style=ButtonStyle())

HTML(value='')

Output()

## Parameter Recommendations

Based on experiments, here are recommended settings for different types of text generation tasks:


In [8]:
recommendations = [
    [
        "Creative Writing",
        "Higher temperature (0.8-1.2), moderate top-p (0.9), low repetition penalty (1.0-1.1)",
    ],
    [
        "Factual Content",
        "Lower temperature (0.3-0.6), lower top-p (0.8), higher repetition penalty (1.2-1.5)",
    ],
    [
        "Code Generation",
        "Lower temperature (0.2-0.5), use top-k (10-40) instead of top-p, high repetition penalty (1.5+)",
    ],
    [
        "Dialogue",
        "Moderate temperature (0.6-0.9), high top-p (0.95), moderate repetition penalty (1.1-1.3)",
    ],
]

display(HTML("<h3>Parameter Recommendations by Task</h3>"))
display(pd.DataFrame(recommendations, columns=["Task Type", "Recommended Parameters"]))

,Task Type,Recommended Parameters
0,Creative Writing,"Higher temperature (0.8-1.2), moderate top-p (..."
1,Factual Content,"Lower temperature (0.3-0.6), lower top-p (0.8)..."
2,Code Generation,"Lower temperature (0.2-0.5), use top-k (10-40)..."
3,Dialogue,"Moderate temperature (0.6-0.9), high top-p (0...."
